In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

將下載好的cuCNN解壓後放在google drive

In [ ]:
%cd /content/

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd /content/darknet

In [ ]:
# 修改 Darknet 設定，符合 Colab 環境
!sed -i "s/GPU=0/GPU=1/g" Makefile
!sed -i "s/CUDNN=0/CUDNN=1/g" Makefile
!sed -i "s/OPENCV=0/OPENCV=1/g" Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# 編譯
!make

In [ ]:
#下載yolov4-tiny的weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

These three functions are helper functions that will allow you to show the image in your Colab Notebook after running your detections, as well as upload and download images to and from your Cloud VM.

In [ ]:
# 將編譯完成的執行檔複製回 Google Drive
!cp -r /content/darknet './gdrive/MyDrive/Colab Notebooks/darknet'
%cd /content

上個步驟會將 darknet 編譯成可以執行的檔案，只有第一次需要做。第二次以後，只要從 Google Drive 將編譯好的檔案複製回 Colab 就可以馬上開始進行訓練了

In [ ]:
!cp -r './gdrive/MyDrive/Colab Notebooks/darknet' ./
!chmod +x ./darknet

Copy collected images and converted labels to darknet/data/obj/

Copy classes.txt in OIDv4_ToolKit to darknet/data/ and rename it as
obj.names

準備要訓練的資料跟環境,這邊使用OpenImages提供的圖片,就不須另外標籤



In [ ]:
!git clone https://github.com/theAIGuysCode/OIDv4_ToolKit

In [ ]:
!pip3 install -r './gdrive/MyDrive/Colab Notebooks/darknet/cuda/requirements.txt'

In [ ]:
%cd /content/OIDv4_ToolKit/

In [ ]:
!pwd

In [ ]:
!python main.py downloader --classes Tomato Carrot Cucumber Cheese Cabbage --type_csv train --limit 100 --multiclasses 1

In [ ]:
!python main.py downloader --classes Tomato Carrot Cucumber Cheese Cabbage --type_csv validation --limit 10 --multiclasses 1

In [ ]:
%cd /content/OIDv4_ToolKit

更改OIDv4_ToolKit裡的classes.txt 改成要train的5個項目,注意空格



In [ ]:
!python convert_annotations.py

In [ ]:
!pwd

In [ ]:
# create the cfg file and Rename yolov4-tiny-custom.cfg 修改裡面的參數
#change line batch to batch=64
#change line subdivisions to subdivisions=16
#change line max_batches to (classes*2000, but not less than number of training images and not less than 6000), f.e. max_batches=6000 if you train for 3 classes
#change line steps to 80% and 90% of max_batches, f.e. steps=4800,5400
#set network size width=416 height=416
#change line classes=80 to your number of objects in each of 3 [yolo]
#change [filters=255] to filters=(classes + 5)x3 in the 3 [convolutional] before each [yolo] layer, keep in mind that it only has to be the last [convolutional] before each of the [yolo] layers.
#when using [Gaussian_yolo] layers, change [filters=57] filters=(classes + 9)x3 in the 3 [convolutional] before each [Gaussian_yolo] layer
#So if classes=1 then should be filters=18. If classes=2 then write filters=21

!cp -r /content/darknet/cfg/yolov4-tiny-custom.cfg  /content/darknet/yolov4-tiny-obj.cfg

在/content/darket/data裡新增一個folder obj
把OIDv4_ToolKit/OID/Dataset/train/Tomato_Carrot_Cucumber_Cheese_Cabbage
OIDv4_ToolKit/OID/Dataset/validation/Tomato_Carrot_Cucumber_Cheese_Cabbage
除了Label的都複製到obj裡

In [ ]:
!cp -r /content/OIDv4_ToolKit/OID/Dataset/train/Tomato_Carrot_Cucumber_Cheese_Cabbage/*.jpg /content/darknet/data/obj/
!cp -r /content/OIDv4_ToolKit/OID/Dataset/train/Tomato_Carrot_Cucumber_Cheese_Cabbage/*.txt /content/darknet/data/obj/

In [ ]:
#把OIDv4_ToolKit/classes.txt 複製到/content/darknet/data/obj.names
!cp /content/OIDv4_ToolKit/classes.txt /content/darknet/data/obj.names

In [ ]:
# 在/content/darknet/data裡新增一個file   obj.data
# classes =5
# train =data/train.txt
# vaild =data/test.txt
# names =data/obj.names
# backup =backup/

YOLOv4模型主要是透過讀取存放照片目錄的txt檔案，來知道要訓練/驗證的照片放置在哪裡。所以此處我們還需要再執行一支程式，產生train.txt及validation.txt兩個照片目錄檔案，供YOLOv4模型來讀取。
我們在C:\Users\User\OIDv4_ToolKit目錄底下，新增這支名為create_catalog.py程式

In [ ]:
# 建立train和valid的照片目錄
import os

# 紀錄當前目錄
originDir = os.getcwd()

# 讀取使用者設定的分類名稱
classes = []
with open("classes.txt", "r") as myFile:
    for num, line in enumerate(myFile, 0):
        line = line.rstrip("\n")
        classes.append(line)
    myFile.close()

# 分類名稱資料夾
classesFolderName = '_'.join(classes)

# 迴圈train和validation資料夾
for folder in ['train', 'validation']:

    # 切換目錄
    os.chdir('./OID/Dataset/' + folder + '/' + classesFolderName)

    # 產生資料夾照片目錄
    image_files = []
    for filename in os.listdir(os.getcwd()):
        if filename.endswith('.jpg'):
            image_files.append('data/' + folder + '/' + filename)

    # 切換至原目錄
    os.chdir(originDir)

    # 寫出照片目錄檔案
    with open(folder + '.txt', 'w') as outfile:
        for image in image_files:
            outfile.write(image)
            outfile.write('\n')
        outfile.close()

In [ ]:
%cd /content/OIDv4_ToolKit/
!cp -r '/content/gdrive/MyDrive/Colab Notebooks/darknet/create_catalog.py' /content/OIDv4_ToolKit

In [ ]:
!python create_catalog.py

In [ ]:
#將使用create_catalog.py產生的txt 複製到 /content/darknet/data/obj 裡
!cp -r /content/OIDv4_ToolKit/train.txt /content/darknet/data
!cp -r /content/OIDv4_ToolKit/validation.txt /content/darknet/data

In [ ]:
!./darknet detector train /content/darknet/data/obj.data /content/darknet/yolov4-tiny-obj.cfg yolov4-tiny.conv.29